### Init Context

In [ ]:
from thetaray.api.context import init_context
import datetime
import yaml

import logging
logging.basicConfig(level=logging.DEBUG, format='%(message)s')

with open('/thetaray/git/solutions/domains/demo_remittance/config/spark_config.yaml') as spark_config_file:
    spark_config = yaml.load(spark_config_file, yaml.FullLoader)['spark_config_a']
context = init_context(execution_date=datetime.datetime(1970, 2, 1),
                       spark_conf=spark_config,
                       spark_master='local[*]')

### Imports

In [ ]:
from thetaray.api.dataset import dataset_functions

from domains.demo_remittance.datasets.customers import customers_dataset
from domains.demo_remittance.datasets.transactions import transactions_dataset
from domains.demo_remittance.datasets.customer_insights import customer_insights_dataset
from domains.demo_remittance.evaluation_flows.ef import evaluation_flow

import json
import psycopg2
import os
import random
import pandas as pd

from datetime import datetime
from faker import Faker
from pyspark.sql import functions as f
from pyspark.sql.types import StructType

from thetaray.api.dataset.schema import DatasetSchemaHandler
from thetaray.common import Constants, Settings
from thetaray.common.data_environment import DataEnvironment

spark = context.get_spark_session()

ns_suffix = Settings.SHARED_NAMESPACE.removeprefix('shared-')

fake = Faker()

### Creation

In [ ]:
DB_HOST = Settings.DB_HOST

DB_USER_CDD = os.environ['CDD_POSTGRES_USERNAME']
DB_PASS_CDD = os.environ['CDD_POSTGRES_PASSWORD']
DB_USER_RP = 'postgres'
DB_PASS_RP = 'postgres'


dsn_cdd = (
    f'user={DB_USER_CDD} '
    f'password={DB_PASS_CDD} '
    f'dbname={Constants.CDD_DB_NAME} '
    f'host={DB_HOST[:-5]} '
    f'port={DB_HOST[-4:]} '
    'sslmode=verify-ca '
    'sslrootcert=/certs/ca.crt'
)


dsn_rp = (
    f'user={DB_USER_RP} '
    f'password={DB_PASS_RP} '
    f'dbname={Constants.CDD_DB_NAME} '
    f'host={DB_HOST[:-5]} '
    f'port={DB_HOST[-4:]} '
    'sslmode=verify-ca '
    'sslrootcert=/certs/ca.crt'
)


def execute_query(query, dsn):
    conn = psycopg2.connect(dsn=dsn)
    with conn.cursor() as cursor:
        cursor.execute(query)
        columns = [col.name for col in cursor.description]
        rows = []
        for row in cursor.fetchall():
            rows.append({col: val for col, val in zip(columns, row)})
        return rows

def get_alert_mapper(solution, ef_id):
    schema = f'apps_{ns_suffix.replace("-", "_")}'
    for alert_mapper in execute_query(f'SELECT * FROM {schema}.rp_mappers', dsn_rp):
        ef_unit = json.loads(alert_mapper['solution_evaluation_flow_unit'])
        if not ef_unit:
            continue
        ef_unit = ef_unit[0]
        if ef_unit['solutionId'] == solution and ef_unit['evaluationFlowId'] == ef_id:
            return alert_mapper


def get_alerts(solution, ef_id):
    schema = f'apps_{ns_suffix.replace("-", "_")}'
    alert_mapper = get_alert_mapper(solution, ef_id)
    if alert_mapper is None:
        raise Exception(f'Alert mapper not found for {solution = } and {ef_id = }')
    alert_mapper_identifier = alert_mapper['identifier']
    alert_fields = execute_query(f'SELECT * FROM {schema}.rp_alert_fields', dsn_rp)
    alert_fields = {alert_field['rp_alert_id']: alert_field for alert_field in alert_fields}
    alerts = execute_query(f"SELECT * FROM {schema}.rp_alerts WHERE alert_mapper_identifier = '{alert_mapper_identifier}' AND history_type = 'CURRENT'", dsn_rp)
    for alert in alerts:
        alert['customer_id'] = alert_fields[alert['alert_id']]['customer_id']
    return alerts


def get_accounts(solution):
    schema = Constants.SOLUTION_SCHEMA_TPL.format(solution=solution)
    query = f"SELECT * FROM {schema}.demo_remittance_customers"
    return execute_query(query, dsn_cdd)


def get_account_records(solution, customer_id):
    schema = Constants.SOLUTION_SCHEMA_TPL.format(solution=solution)
    query = f"SELECT * FROM {schema}.demo_remittance_customers WHERE customer_id = '{customer_id}'"
    return execute_query(query, dsn_cdd)


def get_account_transactions(solution, customer_id):
    schema = Constants.SOLUTION_SCHEMA_TPL.format(solution=solution)
    query = f"SELECT * FROM {schema}.demo_remittance_transactions WHERE customer_id = '{customer_id}'"
    return execute_query(query, dsn_cdd)

In [ ]:
solution = Settings.SOLUTION
ef_id = evaluation_flow().identifier
schema = Constants.SOLUTION_SCHEMA_TPL.format(solution=solution)

alerts = get_alerts(solution, ef_id)
accounts = get_accounts(solution)

account_country = execute_query(f'SELECT * FROM {schema}.demo_remittance_customers', dsn_cdd)
account_country = {a['customer_id']: a['country_of_residence_code'] for a in account_country}

customer_insights_data = []

for account in accounts:
    account_id = account['customer_id']
    account_records = get_account_records(solution, account_id)
    account = sorted(account_records, key=lambda x: x['tr_job_ts'])[-1]  # most recent account data
    account_transactions = get_account_transactions(solution, account_id)
    account_transactions_in = [trx for trx in account_transactions if trx['in_out'] == 'IN']
    account_transactions_out = [trx for trx in account_transactions if trx['in_out'] == 'OUT']
    account_alerts = [alert for alert in alerts if alert['customer_id'] == account_id]

    kyc_classification = 'Medium'
    kyc_name = account['name']
    kyc_is_new = len(account_records) == 1
    kyc_recently_updated = not kyc_is_new
    kyc_newly_incorporation = random.choice([True, False])
    kyc_occupation = account['occupation']
    kyc_null_field = None

    account_transactions_pd = pd.DataFrame(account_transactions)
    hr_cc = [*set(account_transactions_pd.loc[account_transactions_pd.counterparty_country_risk == 'High']['counterparty_country'].tolist())]
    mr_cc = [*set(account_transactions_pd.loc[account_transactions_pd.counterparty_country_risk == 'Medium']['counterparty_country'].tolist())]
    lr_cc = [*set(account_transactions_pd.loc[account_transactions_pd.counterparty_country_risk == 'Low']['counterparty_country'].tolist())]

    director_ad = {'CC': (cc := account['country_of_residence_code']), 'AD': account['address'], 'CL': 'L'}
    director_ad = json.dumps(director_ad)
    company_ad = director_ad
    company_ad = json.dumps(company_ad)

    tr_in = float(sum(trx['amount'] for trx in account_transactions_in))
    tr_out = float(sum(trx['amount'] for trx in account_transactions_out))
    tr_in_count = len(account_transactions_in)
    tr_out_count = len(account_transactions_out)
    tr_in_seg = tr_in * random.uniform(0.4, 0.8)
    tr_out_seg = tr_out * random.uniform(0.4, 0.8)
    tr_in_seg_count = int(tr_in_count * random.uniform(0.4, 0.8))
    tr_out_seg_count = int(tr_out_count * random.uniform(0.4, 0.8))
    trx_from_date = min(map(lambda trx: trx['transaction_timestamp'], account_transactions))
    trx_to_date = max(map(lambda trx: trx['transaction_timestamp'], account_transactions))

    tm_open = len([alert for alert in account_alerts if alert['state_id'] != 'state_closed'])
    tm_closed = len([alert for alert in account_alerts if alert['state_id'] == 'state_closed'])
    tm_false_positives = len([alert for alert in account_alerts if alert['resolution_code'] == 'Non_Issue'])
    tm = {'Open': tm_open, 'Closed': tm_closed, 'False_positives': tm_false_positives}
    tm = json.dumps(tm)

    scrn_open = 0
    scrn_closed = 0
    scrn_false_positives = 0
    scrn = {'Open': scrn_open, 'Closed': scrn_closed, 'False_positives': scrn_false_positives}
    scrn = json.dumps(scrn)

    account_insights_data = {
        'customer_id': account_id,
        'kyc_classification': kyc_classification,
        'kyc_name': kyc_name,
        'kyc_is_new': kyc_is_new,
        'kyc_recently_updated': kyc_recently_updated,
        'kyc_newly_incorporation': kyc_newly_incorporation,
        'kyc_occupation': kyc_occupation,
        'kyc_null_field': kyc_null_field,
        'hr_cc': hr_cc,
        'mr_cc': mr_cc,
        'lr_cc': lr_cc,
        'director_ad': director_ad,
        'company_ad': company_ad,
        'tr_in': tr_in,
        'tr_out': tr_out,
        'tr_in_count': tr_in_count,
        'tr_out_count': tr_out_count,
        'tr_in_seg': tr_in_seg,
        'tr_out_seg': tr_out_seg,
        'tr_in_seg_count': tr_in_seg_count,
        'tr_out_seg_count': tr_out_seg_count,
        'trx_from_date': trx_from_date,
        'trx_to_date': trx_to_date,
        'tm': tm,
        'scrn': scrn
    }

    customer_insights_data.append(account_insights_data)


customer_insights_ds = next(ds for ds in context.solution.datasets if ds.identifier == 'demo_remittance_customer_insights')
customer_insights_schema = DatasetSchemaHandler(customer_insights_ds, context, data_environment=DataEnvironment.get_default())._build_dataset_schema()
customer_insights_schema = StructType([s for s in customer_insights_schema if s.name in [f.identifier for f in customer_insights_ds.field_list]])
customer_insights_df = spark.createDataFrame(customer_insights_data, schema=customer_insights_schema)
customer_insights_df = customer_insights_df.withColumn('tr_timestamp', f.lit(context.execution_date))
customer_insights_df = customer_insights_df.withColumn('effective_date', f.lit(context.execution_date))

In [ ]:
dataset_functions.write(context, customer_insights_df, customer_insights_dataset().identifier)
dataset_functions.publish(context, customer_insights_dataset().identifier)

In [ ]:
context.close()